In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import textblob, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers, Sequential

In [2]:


df = pd.read_csv('L:/Thesis/Dateset/FeaturesExtraction_Compeleted - Binary01.csv', encoding='latin-1')

df = df.astype(dtype={'title': 'string','url': 'string',
                                'date_published': 'string','author_name': 'string',
                                'content': 'string','label': 'string'})

cols = ['countOfPosWord', 'countOfNegWord', 'NumberOfChar','NumberOfWords','NumberOfSentences','AvgCharPerWord','AvgWordPerSent','NumberOfUpCase','NumberOfPunctuatuion','Orgtitle_contentSim','TopicModellingSim','afinn_score','vader_score_compound','FREI','FKGL','ARI','GFI','CLI']
data = pd.read_csv('L:/Thesis/Dateset/FeaturesExtraction_Compeleted - Binary01.csv',  usecols = cols)

In [3]:
df['text_features']= df['title'] +' '+ df['content'] +' '+ df['author_name'] +' '+ df['date_published']
df['Complete_Data'] = df['text_features'] 

In [4]:
df['label'].value_counts()

False    13011
True     12754
Name: label, dtype: Int64

## **SpellCheker**

**Data Preprocessing**

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [5]:
for row in range(len(df)):
  df['Complete_Data'][row] = re.sub(r"won\'t", "will not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"can\'t", "can not", df['Complete_Data'][row])

    # general
  df['Complete_Data'][row] = re.sub(r"n\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'re", " are", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'s", " is", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'d", " would", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ll", " will", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ve", " have", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'m", " am", df['Complete_Data'][row])
print("Done")

Done


In [6]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

X = df['Complete_Data']
stop_words = set(stopwords.words("english"))

import re
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document= [word for word in document if word not in stop_words]
    document = [ps.stem(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
df['Complete_Data'] = documents
print("Done")

Done


**Add Numeric features to text features**

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled = scaler.fit_transform(data)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data_scaled = scaler.fit_transform(data)

In [8]:
numerical_data=list()
for row in range(0,len(df)):
  numerical_data.insert(row,data_scaled[row])
df['numerical_features'] = numerical_data

In [9]:
df = df.astype(dtype={'numerical_features': 'string'})

In [10]:
df['Complete_Data'] = df['Complete_Data'] + df['numerical_features']

In [11]:
df['Complete_Data'][150]

'â everi singl swing seat hous democrat endors medicareforal elect track win elect everi singl one â â unexpect elect loss hous democrat made headlin intern divis parti appeal voter â support progress polici lead member lose seat progress say â rep alexandria ocasio cortez democrat repres part bronx queen said anâ â hous democrat swing district support medicar elect without except â everi singl swing seat hous democrat endors medicareforal elect track win elect everi singl one â everi singl swing seat hous democrat endors elect track win elect everi ð ð ½ singl ð ð ½ one ð ð ½ claim gain traction twitter reader ask us look â gener swing seat competit hous district swing democrat republican control differ organ may rate differ seat competit cook polit report nonpartisan polit analysi organ highli regard commonli cite sourc final nov 2 cook list 23 swing seat held democrat incumb seek elect toss lean democrat 23 two incumb rep matt cartwright pennsylvania peter defazio oregonâ co sponsor

In [12]:
for row in range(0,len(df)):
    df['Complete_Data'][row] = re.sub(r'\[',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\]',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\n',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\s+', ' ', df['Complete_Data'][row], flags=re.I)



## **Set Max features**

In [ ]:
!pip install textstat

In [7]:
import textstat
lenght=list()
for row in range(0,len(df)):
    lenght.insert(row , textstat.lexicon_count(df['Complete_Data'][row]))
df['lenght']=lenght

print('min: ', min(df['lenght']))
print('max: ', max(df['lenght']))

print('mean: {:.2f}'.format(np.mean(df['lenght'])))

print('mean + 2 *sigma: {:.2f}'.format(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght'])))


max_features = int(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght']))
print(max_features)

min:  19
max:  3072
mean: 419.65
mean + 2 *sigma: 918.70
918


In [8]:
encoder = preprocessing.LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

In [9]:
encoder.classes_

array(['False', 'True'], dtype=object)

In [10]:
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(df['Complete_Data'], df['label'], test_size=0.3)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size=0.2)

import tensorflow as tf
train_y = tf.keras.utils.to_categorical(y_train, num_classes=5)
valid_y = tf.keras.utils.to_categorical(y_val, num_classes=5)
test_y = tf.keras.utils.to_categorical(y_test, num_classes=5)

print(len(x_train), len(x_val), len(x_test))
print(len(train_y), len(valid_y), len(test_y))


18035 1546 6184
18035 1546 6184


## **word2vec**

In [11]:
import numpy as np


class Embeddings():
    """
    A class to read the word embedding file and to create the word embedding matrix
    """

    def __init__(self, path, vector_dimension):
        self.path = path 
        self.vector_dimension = vector_dimension
    
    @staticmethod
    def get_coefs(word, *arr): 
        return word, np.asarray(arr, dtype='float32')

    def get_embedding_index(self):
        embeddings_index = dict(self.get_coefs(*o.split(" ")) for o in open(self.path, errors='ignore'))
        return embeddings_index

    def create_embedding_matrix(self, tokenizer, vocab_size):
        """
        A method to create the embedding matrix
        """
        model_embed = self.get_embedding_index()

        embedding_matrix = np.zeros((vocab_size + 1, self.vector_dimension))
        for word, index in tokenizer.word_index.items():
            if index > max_features:
                break
            else:
                try:
                    embedding_matrix[index] = model_embed[word]
                except:
                    continue
        return embedding_matrix


In [12]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(df['Complete_Data'])
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [13]:
embedding = Embeddings(
  'G:/datasets/glove.840B.300d/glove.840B.300d.txt', 
  vector_dimension = 300
)
embedding_matrix = embedding.create_embedding_matrix(tokenizer, vocab_size)

In [14]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
vocab_size = len(tokenizer.word_index) + 1
nonzero_elements / vocab_size

0.008437165571013606

In [15]:
padding_type='post'
trunc_type='post'

#train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(train_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)
#valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(valid_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)

train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=max_features, padding=padding_type, truncating=trunc_type)

test_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=max_features, padding=padding_type, truncating=trunc_type)
valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_val), maxlen=max_features, padding=padding_type, truncating=trunc_type)


In [16]:
print(len(train_seq_x), len(test_seq_x), len(valid_seq_x))


18035 6184 1546


In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/MyDrive/Datasets/Glove.6B/GoogleNews-vectors-negative300.bin',encoding="ISO-8859-1")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float64')
    
#num_word = 10000

# create a tokenizer 
token = text.Tokenizer(oov_token='UNK', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True)
token.fit_on_texts(df['Complete_Data'])
word_index = token.word_index
print(len(word_index))
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
from sklearn.utils import class_weight

def compute_classweights(target):
    """
    Computes the weights of the target values based on the samples
    :param target: Y-target variable
    :return: dictionary object
    """
    # compute class weights
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(target),
                                                     target)
    
    # make the class weight list into dictionary
    weights = {}
    
    # enumerate the list
    for index, weight in enumerate(class_weights):
        weights[index] = weight
        
    return weights

# Get the class weights for the target variable
weights = compute_classweights(df['label'])

C:\Users\Wintest\miniconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1], y=0        0
1        0
2        0
3        0
4        1
        ..
25760    1
25761    1
25762    1
25763    1
25764    1
Name: label, Length: 25765, dtype: int32 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [18]:
weights

{0: 0.9901237414495427, 1: 1.0100752705033715}

In [19]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [20]:
import tensorflow
def scheduler(epoch, lr):
    
    if epoch < 7:
        print('-- lr =%f --',lr) 
        return lr

    elif (epoch%7 == 0):
        lr = lr * tensorflow.math.exp(-0.2)
        print('** lr =%f **',lr)  
        return lr
        
    else:
        print('__ lr =%f __',lr) 
        return lr
    
    
callback = tensorflow.keras.callbacks.LearningRateScheduler(scheduler)

In [22]:
from tensorflow.keras import losses


embedding_dim =300
model_GRU = Sequential()
model_GRU.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_features, weights=[embedding_matrix], trainable=False))
model_GRU.add(layers.BatchNormalization())
model_GRU.add(layers.Bidirectional(layers.GRU(128,return_sequences=True)))
model_GRU.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model_GRU.add(layers.GlobalMaxPool1D()) # Remove flatten layer
model_GRU.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model_GRU.add(layers.Dropout(0.3))
model_GRU.add(layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model_GRU.add(layers.Dropout(0.3))
model_GRU.add(layers.Dense(1, activation='sigmoid'))

model_GRU.summary()

model_GRU.compile(optimizer=tf.keras.optimizers.Adam(8e-6), 
                  loss=losses.BinaryCrossentropy(), 
                  metrics=[tf.metrics.AUC(name='accuracy')])



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 918, 300)          31396800  
_________________________________________________________________
batch_normalization_1 (Batch (None, 918, 300)          1200      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 918, 256)          330240    
_________________________________________________________________
dense (Dense)                (None, 918, 128)          32896     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)               

In [23]:
history = model_GRU.fit(train_seq_x, 
                    y_train,
                    batch_size=4,
                    epochs=50,
                    verbose=1,
                    validation_data=(valid_seq_x, y_val),
                    class_weight=weights,
                    callbacks=[callback])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 18035 samples, validate on 1546 samples
-- lr =%f -- 7.999999979801942e-06
Epoch 1/50
18035/18035 [==============================] - 479s 27ms/sample - loss: 1.2905 - accuracy: 0.6430 - val_loss: 0.9836 - val_accuracy: 0.9252
-- lr =%f -- 7.999999979801942e-06
Epoch 2/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.9593 - accuracy: 0.8917 - val_loss: 0.8429 - val_accuracy: 0.9453
-- lr =%f -- 7.999999979801942e-06
Epoch 3/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.8540 - accuracy: 0.9297 - val_loss: 0.7865 - val_accuracy: 0.9539
-- lr =%f -- 7.999999979801942e-06
Epoch 4/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.7979 - accuracy: 0.9440 - val_loss: 0.7495 - val_accuracy: 0.9597
-- lr =%f -- 7.999999979801942e-06
Epoch 5/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.7581 - accuracy: 0.9530 - val_loss: 0

18035/18035 [==============================] - 470s 26ms/sample - loss: 0.3825 - accuracy: 0.9972 - val_loss: 0.4971 - val_accuracy: 0.9811
** lr =%f ** tf.Tensor(2.409554e-06, shape=(), dtype=float32)
Epoch 43/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.3779 - accuracy: 0.9975 - val_loss: 0.4972 - val_accuracy: 0.9811
__ lr =%f __ 2.409554099358502e-06
Epoch 44/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.3762 - accuracy: 0.9976 - val_loss: 0.4958 - val_accuracy: 0.9816
__ lr =%f __ 2.409554099358502e-06
Epoch 45/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.3729 - accuracy: 0.9978 - val_loss: 0.4942 - val_accuracy: 0.9811
__ lr =%f __ 2.409554099358502e-06
Epoch 46/50
18035/18035 [==============================] - 470s 26ms/sample - loss: 0.3700 - accuracy: 0.9977 - val_loss: 0.4935 - val_accuracy: 0.9811
__ lr =%f __ 2.409554099358502e-06
Epoch 47/50
18035/18035 [=========================

In [24]:

loss, accuracy = model_GRU.evaluate(train_seq_x, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_GRU.evaluate(valid_seq_x, y_val, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

Training Accuracy: 0.9993
Testing Accuracy:  0.9809


In [25]:
model_GRU.evaluate(test_seq_x,y_test,batch_size=4)

6184/6184 [==============================] - 69s 11ms/sample - loss: 0.4864 - accuracy: 0.9818


[0.4864088183541193, 0.9817758]

In [26]:
pred_GRU = model_GRU.predict_classes(test_seq_x)
pred_GRU

array([[0],
       [1],
       [1],
       ...,
       [0],
       [1],
       [1]])

In [27]:
y_eval_GRU = y_test.to_numpy()
y_eval_GRU

array([1, 1, 1, ..., 0, 1, 1])

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix1 = classification_report(y_eval_GRU,pred_GRU)
print(matrix1)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      3101
           1       0.93      0.92      0.92      3083

    accuracy                           0.93      6184
   macro avg       0.93      0.93      0.93      6184
weighted avg       0.93      0.93      0.93      6184



In [29]:
confusion_matrix_gru = confusion_matrix(y_eval_GRU,pred_GRU)
print(confusion_matrix_gru)

[[2902  199]
 [ 262 2821]]
